<a href="https://colab.research.google.com/github/aakarshhh/AI_ML/blob/main/Sent_Stock_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install praw  textblob  nltk yfinance

     |████████████████████████████████| 165 kB 5.2 MB/s 
     |████████████████████████████████| 68 kB 5.7 MB/s 
     |████████████████████████████████| 6.3 MB 51.9 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=06a9cd36a36bc581ce6936a285086e52d6f071511e283e829f4518fbb66af22b
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
import praw
import csv
import re
import json   
import requests
import time
import numpy as np
import pandas as pd
from textblob import TextBlob
import datetime
from re import search
import yfinance as yf
import nltk
from urllib.request import urlopen
import tensorflow as tf
import keras
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import  Dense , Dropout  ,BatchNormalization

from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sia

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
def get_jsonparsed_data(url):
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)
ticker = pd.DataFrame({'Ticker':pd.read_excel('/content/drive/MyDrive/SentStock/tickers.xlsx',header=None)[0],
                       'Company':pd.read_excel('/content/drive/MyDrive/SentStock/Companies.xlsx',header=None)[0] } )
ticker['Buy']=0
ticker['Hold']=0
ticker['Sell']=0

In [4]:
reddit = praw.Reddit( client_secret = 'BnSem-azR1PT-KOweDLrTDx_GoOOYA',
 client_id = 'bpxOQpC_a3i8xA',
 user_agent = 'StonksSent',
 username = 'LaLaA80524',
 password = 'SentStock')
subr= reddit.subreddit('wallstreetbets+robinhood+algotrading+investing+stocks+SecurityAnalysis+StockMarket')
t = datetime.datetime.now()
t = t- datetime.timedelta(days = 7)
print(t)

2021-07-16 06:55:58.317028


In [5]:
param = pd.read_csv('/content/drive/MyDrive/SentStock/param.csv',header=0).rename(index={0:'mean',1:'std',2:'pred', 3:'real',4:'pred_next'})
param = param.loc[['mean','std','pred']]

In [6]:
a = pd.read_excel(f'/content/drive/MyDrive/SentStock/preds_{t.strftime("%m_%d_%Y")}.xlsx',header=0)
a.rename(columns={'Unnamed: 0':0},inplace= True)
a.set_index(0,inplace= True)
a = a.loc[['pred_next6']]
t = t+ datetime.timedelta(days = 1)

In [7]:
b = pd.read_excel(f'/content/drive/MyDrive/SentStock/preds_{t.strftime("%m_%d_%Y")}.xlsx',header=0)
b.rename(columns={'Unnamed: 0':0},inplace= True)
b.set_index(0,inplace= True)
b = b.loc[['pred_next5']]
t = t+ datetime.timedelta(days = 1)

In [8]:
cd = pd.read_excel(f'/content/drive/MyDrive/SentStock/preds_{t.strftime("%m_%d_%Y")}.xlsx',header=0)
cd.rename(columns={'Unnamed: 0':0},inplace= True)
cd.set_index(0,inplace= True)
cd = cd.loc[['pred_next4']]
t = t+ datetime.timedelta(days = 1)

In [9]:
d = pd.read_excel(f'/content/drive/MyDrive/SentStock/preds_{t.strftime("%m_%d_%Y")}.xlsx',header=0)
d.rename(columns={'Unnamed: 0':0},inplace= True)
d.set_index(0,inplace= True)
d = d.loc[['pred_next3']]
t = t+ datetime.timedelta(days = 1)

In [10]:
e = pd.read_excel(f'/content/drive/MyDrive/SentStock/preds_{t.strftime("%m_%d_%Y")}.xlsx',header=0)
e.rename(columns={'Unnamed: 0':0},inplace= True)
e.set_index(0,inplace= True)
e = e.loc[['pred_next2']]
t = t+ datetime.timedelta(days = 1)

In [11]:
f = pd.read_excel(f'/content/drive/MyDrive/SentStock/preds_{t.strftime("%m_%d_%Y")}.xlsx',header=0)
f.rename(columns={'Unnamed: 0':0},inplace= True)
f.set_index(0,inplace= True)
f = f.loc[['pred_next']]
t = t+ datetime.timedelta(days = 1)

In [12]:
g = pd.read_excel(f'/content/drive/MyDrive/SentStock/preds_{t.strftime("%m_%d_%Y")}.xlsx',header=0)
g.rename(columns={'Unnamed: 0':0},inplace= True)
g.set_index(0,inplace= True)
g = g.loc[['pred']]
t = t+ datetime.timedelta(days = 1)

In [13]:
top_stuff=subr.top(time_filter='week' ,limit=5)
for submission in top_stuff:
  submission.comments.replace_more(limit=0)
  comments = submission.comments.list() 
  for word in word_tokenize(submission.title):
    for i in range(len(ticker)):
      if word == '$'+ str(ticker['Ticker'][i]) or word == str(ticker['Ticker'][i]) or str(ticker['Company'][i]).find(word)!=-1:
        buy_upvote = 0 
        sell_upvote = 0 
        hold_upvote = 0 
        comments = submission.comments.list()
        for comment in comments :
         if comment.created_utc >= t.replace(tzinfo=datetime.timezone.utc).timestamp():
           buy_list = re.findall(r'\bbuy\b',comment.body, flags = re.IGNORECASE )
           sell_list = re.findall(r'\bsell\b',comment.body, flags = re.IGNORECASE )
           hold_list = re.findall(r'\bhold\b',comment.body, flags = re.IGNORECASE )
           d_buy_list = re.findall(r"\bdon't buy\b",comment.body, flags = re.IGNORECASE )
           d_sell_list = re.findall(r"\bdon't sell\b",comment.body, flags = re.IGNORECASE )
           d_hold_list = re.findall(r"\bdon't hold\b",comment.body, flags = re.IGNORECASE )
           if d_buy_list ==[]:
             if buy_list != []:
               buy_upvote +=comment.ups
           if sell_list == []:
             _
           elif d_sell_list == []:
             sell_upvote +=comment.ups
           else:
             hold_upvote+=comment.ups
    
           if hold_list == []:
             _
           elif d_hold_list == []:
             hold_upvote+=comment.ups
           else:
             sell_upvote += comment.ups
        ticker['Buy'][i]+=buy_upvote/len(word_tokenize(ticker['Company'][i]))
        ticker['Hold'][i]+=hold_upvote/len(word_tokenize(ticker['Company'][i]))
        ticker['Sell'][i]+=sell_upvote/len(word_tokenize(ticker['Company'][i]))


It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [14]:
ticker.set_index('Ticker',inplace=True)
ticker[['Buy','Hold','Sell']]=ticker[['Buy','Hold','Sell']].div( ticker[['Buy','Hold','Sell']].sum(axis = 1), axis = 'index')
ticker.fillna(0,inplace= True)
t = datetime.datetime.now()
ticker.to_excel(f'/content/drive/MyDrive/SentStock/praw_{t.strftime("%m_%d_%Y")}.xlsx')

In [15]:
ticker

,Company,Buy,Hold,Sell
Ticker,,,,
T,AT&T,0.0,0.0,0.0
USM,US Cellular,0.0,0.0,0.0
VZ,Verizon,0.0,0.0,0.0
AAPL,Apple,0.0,0.0,0.0
MSFT,Microsoft,0.0,0.0,0.0
...,...,...,...,...
BUD,Anheuser-Busch,0.0,0.0,0.0
PM,Phillip Morris International,0.0,0.0,0.0
PEP,Pepsico,0.0,0.0,0.0


In [16]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
from numpy import arange
def train_model(x,y,ep = 7):
  model.fit(x,y, epochs=11 , batch_size = 1 , verbose=2 )

def norm(x , c ):
  sc.mean_= param[f'{c}']['mean']
  sc.scale_= param[f'{c}']['std']
  return sc.transform(x)

def scaled(y , c ):
  sc.mean_= param[f'{c}']['mean']
  sc.scale_= param[f'{c}']['std']
  return sc.inverse_transform(y)

In [17]:
model= Sequential()
model.add(Dense(22, activation='relu',input_shape= (1,10)))
model.add(Dense(44, activation='relu'))
model.add(Dense(11, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(units = 1))
model.compile(loss = 'mse' , optimizer = 'adam' )

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 22)             242       
_________________________________________________________________
dense_1 (Dense)              (None, 1, 44)             1012      
_________________________________________________________________
dense_2 (Dense)              (None, 1, 11)             495       
_________________________________________________________________
dense_3 (Dense)              (None, 1, 4)              48        
_________________________________________________________________
dense_4 (Dense)              (None, 1, 1)              5         
Total params: 1,802
Trainable params: 1,802
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.load_weights('/content/drive/MyDrive/SentStock/stock_sent.keras')

In [ ]:
count = 1 
for c in pd.read_excel('/content/drive/MyDrive/SentStock/tickers.xlsx',header=None)[0]:
  print("Training started for "+c)
  c = c.strip()
  xt = np.array([a[f'{c}'].values[0] ,b[f'{c}'].values[0],cd[f'{c}'].values[0],d[f'{c}'].values[0] , e[f'{c}'].values[0] , f[f'{c}'].values[0] , g[f'{c}'].values[0] , ticker[['Buy','Hold','Sell']].loc[f'{c}'].values[0], ticker[['Buy','Hold','Sell']].loc[f'{c}'].values[1], ticker[['Buy','Hold','Sell']].loc[f'{c}'].values[2]]).reshape(-1,1)  
  param[f'{c}']['mean'] = xt.mean()
  param[f'{c}']['std'] = xt.std() 
  xt = norm(xt,c).reshape(1,-1)
  print(xt.shape)
  data = yf.download(
      
        tickers = f'{c}',
        start =t-datetime.timedelta(days = 2), end=t,
        group_by = 'ticker',
        auto_adjust = True,
        prepost = True
        
  )
  data = pd.DataFrame(data)
  if data.shape[0] == 0 :
    pass
  yt = norm(data[-1:]['Close'].values.reshape(1, -1), c )[0]
  
  train_model(xt,yt)
  param[f'{c}']['pred'] = scaled(model.predict(xt),c)
  print(count)
  count+=1

Training started for T
(1, 10)
[*********************100%***********************]  1 of 1 completed
Epoch 1/11
1/1 - 1s - loss: 0.0084
Epoch 2/11
1/1 - 0s - loss: 0.0059
Epoch 3/11
1/1 - 0s - loss: 0.0039
Epoch 4/11
1/1 - 0s - loss: 0.0024
Epoch 5/11
1/1 - 0s - loss: 0.0013
Epoch 6/11
1/1 - 0s - loss: 5.4641e-04
Epoch 7/11
1/1 - 0s - loss: 1.4122e-04
Epoch 8/11
1/1 - 0s - loss: 2.2297e-06
Epoch 9/11
1/1 - 0s - loss: 5.8153e-05
Epoch 10/11
1/1 - 0s - loss: 2.3513e-04
Epoch 11/11
1/1 - 0s - loss: 4.6399e-04
1
Training started for USM
(1, 10)
[*********************100%***********************]  1 of 1 completed
Epoch 1/11
1/1 - 0s - loss: 0.0027
Epoch 2/11
1/1 - 0s - loss: 0.0029
Epoch 3/11
1/1 - 0s - loss: 0.0029
Epoch 4/11
1/1 - 0s - loss: 0.0027
Epoch 5/11
1/1 - 0s - loss: 0.0024
Epoch 6/11
1/1 - 0s - loss: 0.0020
Epoch 7/11
1/1 - 0s - loss: 0.0016
Epoch 8/11
1/1 - 0s - loss: 0.0011
Epoch 9/11
1/1 - 0s - loss: 7.5501e-04
Epoch 10/11
1/1 - 0s - loss: 4.3500e-04
Epoch 11/11
1/1 - 0s - los

In [ ]:
param

In [ ]:
param.to_csv(f'/content/drive/MyDrive/SentStock/sent_{t.strftime("%m_%d_%Y")}.csv')
model.save('/content/drive/MyDrive/SentStock/stock_sent.keras')